In [2]:
!pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ----- ---------------------------------- 1.6/12.0 MB 7.0 MB/s eta 0:00:02
   ------------- -------------------------- 3.9/12.0 MB 9.0 MB/s eta 0:00:01
   ----------------------- ---------------- 7.1/12.0 MB 10.9 MB/s eta 0:00:01
   --------------------------------- ------ 10.0/12.0 MB 11.7 MB/s eta 0:00:01
   ---------------------------------------  11.8/12.0 MB 11.5 MB/s eta 0:00:01
   ---------------------------------------- 12.0/12.0 MB 10.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/566.1 kB ? eta -:--:--
   ---------------------------------------- 566.1/566.1 kB 4.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   --------------------------- ------------ 1.8/2.7 MB 11.2 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 8.6 MB/s eta 0:00:00
   -------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install faiss-cpu

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/18.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.8 MB ? eta -:--:--
    --------------------------------------- 0.3/18.8 MB ? eta -:--:--
   - -------------------------------------- 0.5/18.8 MB 2.4 MB/s eta 0:00:08
   - -------------------------------------- 0.8/18.8 MB 1.7 MB/s eta 0:00:11
   -- ------------------------------------- 1.0/18.8 MB 1.5 MB/s eta 0:00:12
   -- ------------------------------------- 1.3/18.8 MB 1.4 MB/s eta 0:00:13
   --- ------------------------------------ 1.6/18.8 MB 1.2 MB/s eta 0:00:14
   --- ------------------------------------ 1.8/18.8 MB 1.2 MB/s eta 0:00:14
   --- ------------------------------------ 1.8/18.8 MB 1.2 MB/s eta 0:00:14
   ---- ----------------------------------- 2.1/18.8 MB 1.2 MB/s eta 0:00:15
   ----- ---------------------------------- 2.4/18.8 MB 1.1 MB/s eta 0:00:15
   ----- ----------


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import os
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss

In [6]:
RAW_DIR = "../data/raw"
PROCESSED_DIR = "../data/processed"
MODEL_DIR = "../data/models/reco_builder"
os.makedirs(MODEL_DIR, exist_ok=True)


In [7]:
# Charger les données
resources_file = os.path.join(PROCESSED_DIR, "learning_resources.csv")
if os.path.exists(resources_file):
    resources_df = pd.read_csv(resources_file)
else:
    # Exemple de ressources fictives
    data = {
        "resource_id": [1, 2, 3, 4, 5],
        "title": [
            "Introduction to Module AAA Concepts",
            "Quiz on Key Concepts",
            "Module AAA Study Notes",
            "Advanced Topic Deep Dive",
            "Video: Module AAA Overview"
        ],
        "type": ["video", "exercise", "pdf", "tutoring", "video"],
        "topic": ["AAA Basics", "AAA Basics", "AAA Notes", "AAA Advanced", "AAA Overview"],
        "difficulty": ["medium", "high", "medium", "high", "low"]
    }
    resources_df = pd.DataFrame(data)

print("✔️ Ressources chargées :")
display(resources_df)

✔️ Ressources chargées :


,resource_id,title,type,topic,difficulty
0,1,Introduction to Module AAA Concepts,video,AAA Basics,medium
1,2,Quiz on Key Concepts,exercise,AAA Basics,high
2,3,Module AAA Study Notes,pdf,AAA Notes,medium
3,4,Advanced Topic Deep Dive,tutoring,AAA Advanced,high
4,5,Video: Module AAA Overview,video,AAA Overview,low


In [8]:
# Charger le profil étudiant (ex. résultats + engagement)
student_metrics_file = os.path.join(PROCESSED_DIR, "student_module_metrics.csv")
if os.path.exists(student_metrics_file):
    student_metrics = pd.read_csv(student_metrics_file)
else:
    # Exemple de profil étudiant
    student_metrics = pd.DataFrame([{
        "student_id": 11391,
        "weak_topics": "AAA Advanced AAA Basics",
        "engagement_level": "medium"
    }])

In [11]:
def get_weak_topics(row):
    topics = []
    if row['avg_score'] < 50:  # seuil à ajuster selon ton besoin
        topics.append(row['code_module'])
    if row['completion_rate'] < 0.5:
        topics.append(row['code_module'])
    return " ".join(topics) if topics else "None"

student_metrics['weak_topics'] = student_metrics.apply(get_weak_topics, axis=1)

# Encodage BERT
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encodage des ressources
resource_texts = (resources_df["title"] + " " + resources_df["topic"]).tolist()
resource_embeddings = model.encode(resource_texts, convert_to_numpy=True)
print("✔️ Ressources encodées.")

# Encodage du profil étudiant
def encode_student_profile(row):
    return model.encode([row["weak_topics"]], convert_to_numpy=True)[0]

student_metrics["embedding"] = student_metrics.apply(encode_student_profile, axis=1)
print("✔️ Profils étudiants encodés.")


✔️ Ressources encodées.
✔️ Profils étudiants encodés.


In [12]:
# Indexation Faiss
embedding_dim = resource_embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(resource_embeddings)
print(f"✔️ Index Faiss créé avec {index.ntotal} vecteurs.")

✔️ Index Faiss créé avec 5 vecteurs.


In [13]:
def recommend_resources(student_id, top_k=3):
    student_row = student_metrics[student_metrics["student_id"] == student_id]
    if student_row.empty:
        print("❌ Étudiant introuvable")
        return
    
    student_vector = np.array([student_row.iloc[0]["embedding"]])
    distances, indices = index.search(student_vector, top_k)
    
    print(f"🎯 Top-{top_k} recommandations pour l'étudiant {student_id} :")
    for rank, idx in enumerate(indices[0]):
        res = resources_df.iloc[idx]
        print(f"{rank+1}. {res['type'].capitalize()}: {res['title']} (Topic: {res['topic']}, Difficulty: {res['difficulty']})")


In [ ]:
recommend_resources(11391, top_k=3)

🎯 Top-3 recommandations pour l'étudiant 11391 :
1. Exercise: Quiz on Key Concepts (Topic: AAA Basics, Difficulty: high)
2. Video: Video: Module AAA Overview (Topic: AAA Overview, Difficulty: low)
3. Tutoring: Advanced Topic Deep Dive (Topic: AAA Advanced, Difficulty: high)
